<a href="https://colab.research.google.com/github/tylerle29/Memotron/blob/main/Copy_of_NVIDIA_Nemotron_Nano_9B_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install -U transformers

trust_remote_code=True

In [13]:
!pip install -U mamba-ssm

  Using cached mamba_ssm-2.2.6.post3.tar.gz (113 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached ninja-1.13.0-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (5.1 kB)
Using cached ninja-1.13.0-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (180 kB)
  Created wheel for mamba-ssm: filename=mamba_ssm-2.2.6.post3-cp312-cp312-linux_x86_64.whl size=370558504 sha256=6621f321a90889485cda036f2d8881f254222faa4f37042c00a2f8a4e191500a
  Stored in directory: /root/.cache/pip/wheels/a9/66/b0/91003b49a34b45995d1b34c7fd2c70cd998a23560e66f6817a
Successfully built mamba-ssm


## Local Inference on GPU
Model page: https://huggingface.co/nvidia/NVIDIA-Nemotron-Nano-9B-v2

⚠️ If the generated code snippets do not work, please open an issue on either the [model repo](https://huggingface.co/nvidia/NVIDIA-Nemotron-Nano-9B-v2)
			and/or on [huggingface.js](https://github.com/huggingface/huggingface.js/blob/main/packages/tasks/src/model-libraries-snippets.ts) 🙏

In [ ]:
'''
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("nvidia/NVIDIA-Nemotron-Nano-9B-v2")

# Load model (on GPU if available)
device = "cuda" if torch.cuda.is_available() else "cpu"
model = AutoModelForCausalLM.from_pretrained(
    "nvidia/NVIDIA-Nemotron-Nano-9B-v2",
    torch_dtype=torch.bfloat16,  # saves memory, works on GPU
    trust_remote_code=True,
    device_map="auto"
)

# Example text input
input_text = "Hello, how are you today?"

# Tokenize input
inputs = tokenizer(input_text, return_tensors="pt").to(device)

# Generate output (limited to 50 tokens)
outputs = model.generate(**inputs, max_new_tokens=50)

# Decode output
decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("Input:", input_text)
print("Output:", decoded_output)
'''

In [11]:
!pip install flask flask-ngrok transformers torch -q

from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [12]:
app = Flask(__name__)
run_with_ngrok(app)  # exposes public URL

# -------------------------------
# Load Nemotron-Nano-9B-v2
# -------------------------------
device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained("nvidia/NVIDIA-Nemotron-Nano-9B-v2")
model = AutoModelForCausalLM.from_pretrained(
    "nvidia/NVIDIA-Nemotron-Nano-9B-v2",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto"
)

# -------------------------------
# Flask route for React frontend
# -------------------------------
@app.route("/api/respond", methods=["POST"])
def respond():
    data = request.json
    input_text = data.get("input", "")

    # Tokenize input
    inputs = tokenizer(input_text, return_tensors="pt").to(device)

    # Generate output (50 tokens max)
    outputs = model.generate(**inputs, max_new_tokens=50)

    # Decode output
    decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return jsonify({
        "input": input_text,
        "output": decoded_output
    })

# -------------------------------
# Run Flask app
# -------------------------------
app.run()


ImportError: mamba-ssm is required by the Mamba model but cannot be imported

In [ ]:
# Clear the CUDA cache
torch.cuda.empty_cache()

# Delete variables that might be holding onto GPU memory
# You might need to add other variable names here if you have more objects on the GPU
if 'model' in locals():
    del model
if 'pipe' in locals():
    del pipe
if 'pipe_smaller' in locals():
    del pipe_smaller
if 'input_ids' in locals():
    del input_ids
if 'outputs' in locals():
    del outputs
if 'tokenizer' in locals():
    del tokenizer

# Run Python's garbage collector
import gc
gc.collect()

print("GPU memory hopefully cleared.")